## Produce daily Origin-Destination trip matrix for NYC resident cbgs

##### Can aggregate to different NYC geo summary levels and destination geo summary levels, separates visits to home cbg
##### Output is daily csv o-d matrices

In [1]:
import pandas as pd
import numpy as np
import s3fs
import os
import time

In [2]:
from geo import stco,sub
from safegraph_py_functions import safegraph_py_functions as sgpy

In [3]:
%load_ext dotenv
%dotenv
myAccessKey = os.getenv('myAccessKey')
mySecretKey = os.getenv('mySecretKey')

start = time.time()

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [5]:
# read nyc origin cbgs
cbg_nyc = pd.read_csv(f'../data/nyc_cbg.csv')

In [6]:
# specify the SG key and secret
fs = s3fs.S3FileSystem(profile="safegraphws", key=myAccessKey, secret=mySecretKey, client_kwargs={'endpoint_url': 'https://s3.wasabisys.com', 'region_name':'us-east-1'})

In [7]:
# set date variables, in this case, a test month for 2 different years
month = "01"
years = ["2019","2020"]
dayList =["01","02","03","04","05","06","07","08","09","10","11","12","13","14","15","16","17","18","19","20","21","22","23","24","25","26","27","28","29","30","31"]

# If running all days and months - replace range and adjust for loop to run through lists below:
#monthList =["01","02","03","04","05","06","07","08"]
#dayNumList =[31, 29, 31, 30, 31, 30, 31,31] 

In [8]:
## Test month - individual days for year-over-year comparison

frames = []
for y in years:
    for i in range(5,8):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{month}-{dayList[i]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            #unpack json destination cbgs
            df = sgpy.unpack_json_and_merge(df, json_column='destination_cbgs', key_col_name='destination_cbg', value_col_name='dest_cbg_count')

            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            df['date_y-m-d'] = df['date_range_start'].str[:10]
            #separate home trips from other trips for later aggregation
            df['is_home'] = df.apply(lambda x: x['orig_cbg']==x['destination_cbg'],axis=1)
            #id destinations by county fips
            df['dest_stco_all'] = df['destination_cbg'].str[:5]
            #id 31cr counties and all others outside
            df['dest_stco_reg'] = df['dest_stco_all'] 
            df.loc[~df['dest_stco_reg'].isin(stco),'dest_stco_reg'] = 'O31CR' 
            df['dest_sub'] = df['dest_stco_all'].map(sub).fillna(df['dest_stco_all'].str[:2])

            #Make new table with select columns
            dff = df[['date_y-m-d','orig_cbg','dest_cbg_count','dest_stco_reg','is_home']]
            frames.append(dff)
            

2019-01-06
2019-01-07
2019-01-08
2020-01-06
2020-01-07
2020-01-08


In [9]:
df_dest = pd.concat(frames)
df_dest = pd.pivot_table(df_dest,values=['dest_cbg_count'],index=['date_y-m-d','orig_cbg'],columns=['dest_stco_reg','is_home'],aggfunc=np.sum,\
                         fill_value=0,margins=True)

In [19]:
## Add in total device and completely home device counts
frames = []
for y in years:
    for i in range(5,8):
        with fs.open(f'sg-c19-response/social-distancing/v2/{y}/{month}/{dayList[i]}/{y}-{month}-{dayList[i]}-social-distancing.csv.gz','rb') as f:
            print(f'{y}-{month}-{dayList[i]}')
            # read SG's file
            df = pd.read_csv(f, escapechar='\\', compression='gzip')
            # filter NYC's Origin CBGs
            df = pd.merge(cbg_nyc, df, left_on="orig_cbg", right_on="origin_census_block_group", how="inner")

            ##Make new columns
            df['orig_cbg'] = df['orig_cbg'].apply(str) #clean origin cbg
            df['date_y-m-d'] = df['date_range_start'].str[:10]

            #Make new table with select columns
            dff = df[['date_y-m-d','orig_cbg','device_count','completely_home_device_count']]
            frames.append(dff)
            

2019-01-06
2019-01-07
2019-01-08
2020-01-06
2020-01-07
2020-01-08


In [21]:
df_device = pd.concat(frames)

In [22]:
df_device.head()

,date_y-m-d,orig_cbg,device_count,completely_home_device_count
0,2019-01-06,360050001001,13,1
1,2019-01-06,360050002001,79,47
2,2019-01-06,360050002002,142,77
3,2019-01-06,360050002003,93,46
4,2019-01-06,360050004000,32,7


In [24]:
df_device = df_device.groupby(['date_y-m-d','orig_cbg']).agg({'device_count':np.sum,'completely_home_device_count':np.sum})

In [25]:
df_master = pd.concat([df_device,df_dest],axis=1)
df_master = df_master.reset_index()

In [26]:
df_master.to_excel(f'output/origdestcbg_check.xlsx')

In [13]:
end = time.time()
elapsed = end - start
print(f'Run time - {elapsed} seconds')

Run time - 615.2352719306946 seconds
